#Lab.10 / IBM3202 – Conformational changes using Structure-Based Models

###Theoretical aspects

The **energy landscape theory** and the **principle of minimum frustration** in protein folding have provided the theoretical basis for the generation of simplified models to simulate the pathways of protein folding of different proteins. Noteworthy, recent work has demonstrated their utility for simulating other functionally relevant phenomena, such as **protein misfolding and conformational changes** associated to biological function. Most of these applications have been generated through savvy and careful combinations of the native bonded and non-bonded terms from **two or more structures deposited in the PDB** in two or more conformational states (i.e. open and closed conformations, alpha and beta states, etc).

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/smogdual_01.png'/>
<figcaption>FIGURE 1. Modeling the conformational transitions of adenylate kinase. This enzyme undergoes a >25 Å motion between open (red, PDB 4AKE) and closed (green, PDB 1AKE) states due to ligand binding. The relative occupation of the closed and open states can be tuned to experimental data by varying the strength of the subset of contacts only existing in the closed state between 0.6 (red) to 1.2 (black) relative to the open contacts <br> Noel JK & Onuchic JN (2012) <i> Computational Modeling of Biological Systems, 31-54</i></figcaption></center>
</figure>

##Experimental Overview

In this tutorial we will exemplify how we can **combine the native contacts of two different structures** to simulate the conformational change of adenylate kinase, an enzyme that has three domains (LID, NMP and core) and catalyzes the phosphorylation reaction of AMP, using ATP, generating 2 molecules of ADP as product:

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/smogdual_02.png'/>
</center>
</figure>

This reaction requires a severe conformational change, which can be seen from the structures of the protein in the presence (1AKE) and in the absence (4AKE) of substrates.

#Part 0. Downloading and Installing the required software

Before we start, **remember to start the hosted runtime** in Google Colab.

Then, we must install several pieces of software to perform this tutorial. Namely:
- **biopython** for manipulation of the PDB files
- **py3Dmol** for visualization of the protein structure.
- **udocker** for installing and running Docker containers
- **SMOG2** for generating our structure-based models
- **SBM-enhanced GROMACS** for preparing our MD system and performing our MD simulations.

For visualizing our MD trajectories, we will employ a web version of **NGLview**. This is due to the inability of Google Colab to handle a required python package for loading NGLview directly onto Google Colab. Hopefully this will change in the near future.

1. First, we will start by installing **udocker** to pull and run a Docker version of **SMOG2** onto Google Colab.

In [ ]:
#Downloading udocker and installing smog2
!pip install udocker
!udocker --allow-root install
!udocker --allow-root pull smogserver/smog2

In [ ]:
#Test the smog2 docker installation
!udocker --allow-root run --rm -v $(pwd):/workdir smogserver/smog2:stable smog2 --help

2. Then, we will set up our SBM-enhanced GROMACS on Google Colab, based on your previously compiled and installed GROMACS

In [ ]:
# Download and unzip the compressed folder of SBM-enhanced GROMACS
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/software/gromacs_sbm.tar.gz
!tar xzf gromacs_sbm.tar.gz

3. Lastly, we will install biopython and py3Dmol

In [ ]:
!pip install biopython py3dmol

Once these software installation processes are completed, we are ready to perform our experiments

# Part I – Generate coarse-grained SBM models using SMOG2

As we did in the previous tutorial, we will first download the coordinates for the solved structures of human adenylate kinase in the open (4AKE) and closed (1AKE) states to generate coarse-grained SBMs for both structures using SMOG2. Then, we will employ a combination of numerical analysis and SMOG2 to generate custom-built SBM models that contain information from both structures in a single file (**dual-basin models**).

1. We start by creating and accessing a folder for preparing our systems

In [ ]:
#Let's make a folder first. We need to import the os and path library
import os
from pathlib import Path

#Then, we define the path of the folder we want to create.
#Notice that the HOME folder for a hosted runtime in colab is /content/
smogpath = Path("/content/prepare_dualAKE/")

#Now, we create the folder using the os.mkdir() command
#The if conditional is just to check whether the folder already exists
#In which case, python returns an error
if os.path.exists(smogpath):
  print("path already exists")
if not os.path.exists(smogpath):
  os.mkdir(smogpath)
  print("path was succesfully created")

In [ ]:
#Changing directory using python
os.chdir(smogpath)

2. Then, we will download the solved structures of human adenylate kinase in the open (PDB 4AKE) and closed (PDB 1AKE) conformations, and remove alternative side chain conformations, water molecules and ligands using biopython as we have done in our previous tutorials.

**NOTE:** You might get a _chain discontinuity_ warning on biopython. In this particular case, this is due to the non-contiguous annotation of non-protein atoms from chain A and B in the PDB file.

In [ ]:
#Importing your PDB file using biopython
import os
from Bio.PDB import *
pdbid = ['1ake', '4ake']
pdbl = PDBList()
for s in pdbid:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
  os.rename("pdb"+s+".ent", s+".pdb")

In [ ]:
#Here we set up a parser for our PDB
parser = PDBParser()
io=PDBIO()

#And here we set the residue conformation we want to keep
keepAltID = "A"
class KeepOneConfOnly(Select):  # Inherit methods from Select class
    def accept_atom(self, atom):
        if (not atom.is_disordered()) or atom.get_altloc() == keepAltID:
            atom.set_altloc(" ")  # Eliminate alt location ID before output.
            return True
        else:  # Alt location was not one to be output.
            return False
        # end of accept_atom()

#And now we loop for all structures
for s in pdbid:
  structure = parser.get_structure('X', s+".pdb")
  #This will keep only conformation for each residue
  io.set_structure(structure)
  io.save(s+"_ready.pdb", select=KeepOneConfOnly())
print("Your PDBs were processed. Alternative side chain conformations removed!")

In [ ]:
#Here we set up a parser for our PDB
parser = PDBParser()
io=PDBIO()
for s in pdbid:
  structure = parser.get_structure('X', s+"_ready.pdb")
  #And here we remove hydrogens, waters and ligands using Dice
  io.set_structure(structure)
  sel = Dice.ChainSelector('A', 1, 214)
  io.save(s+"_clean.pdb", sel)
print("Your PDBs were processed. Only the protein heavy atoms have been kept!")

3. Let's examine our structures in py3Dmol

In [ ]:
import py3Dmol
#First we assign the py3Dmol.view as a two-panel viewer
view=py3Dmol.view(800,400)
#Here we set the background color as white
view.setBackgroundColor('white')
#The following lines are used to add the addModel class
#to read the open state structure
view.addModel(open('4ake_clean.pdb', 'r').read(),'pdb')
#Here we set the visualization style and color
view.setStyle({'chain':'A'},{'cartoon': {'color':'red'}})
#Here we center the molecule for its visualization
view.zoomTo()
#And we finally visualize the structures using the command below
view.show()

In [ ]:
import py3Dmol
#First we assign the py3Dmol.view as a two-panel viewer
view=py3Dmol.view(800,400)
#Here we set the background color as white
view.setBackgroundColor('white')
#Now we do the same for the closed state structure
view.addModel(open('1ake_clean.pdb', 'r').read(),'pdb')
#Here we set the visualization style and color
view.setStyle({'chain':'A'},{'cartoon': {'color':'green'}})
#Here we center the molecule for its visualization
view.zoomTo()
#And we finally visualize the structures using the command below
view.show()

4. As we saw in our previous tutorial, these PDB files need to be adjusted to fit the format requirements of SMOG2. We will make these adjustments using the **smog_adjustPDB** module of SMOG2.

In [ ]:
!udocker --allow-root run --rm -v $(pwd):/workdir smogserver/smog2:stable smog_adjustPDB -i 4ake_clean.pdb -o 4ake_adjusted.pdb
!udocker --allow-root run --rm -v $(pwd):/workdir smogserver/smog2:stable smog_adjustPDB -i 1ake_clean.pdb -o 1ake_adjusted.pdb

5. Once this is done, we can process our files in SMOG2 as indicated below:

In [ ]:
!udocker --allow-root run --rm -v $(pwd):/workdir smogserver/smog2:stable smog2 -i 4ake_adjusted.pdb -CA -dname 4ake_smog
!udocker --allow-root run --rm -v $(pwd):/workdir smogserver/smog2:stable smog2 -i 1ake_adjusted.pdb -CA -dname 1ake_smog

#Part II – Generate a custom dual-basin SBM model using SMOG2

We should now have our SBM models for the open and the closed state of human adenylate kinase. However, if you remember from our lectures and our previous tutorial, these coarse-grained models do not contain water molecules, ligands, etc. Moreover, since we do not have water in our system, the inclusion of a ligand will lead to its drift outside the active site towards infinity (as you can see from the .mdp simulation file, we are not using periodic boundary conditions).

**How are we going to simulate a conformational change in the absence of ligands?**

Instead of thinking of including the ligand, we must think about the **consequences of ligand binding**. In this case, ligand binding leads to the three domains of the protein getting closer to each other, which translates into **several native contacts being formed upon ligand binding** (or unique to the closed conformation).

Briefly, to simulate the conformational change of this enzyme, it is most appropriate to consider that:
- The **open state** is the **initial condition**, since it can exist both in the absence of and at low concentrations of ligand;
- The native interactions that are **unique to the closed state** correspond to **ligand-induced interactions**;
- The **native ligand-induced interactions** that exhibit significant changes in distance (eg. dist [4AKE] / dist [1AKE] > 50%) are **the only ones to be included in a combined native contact map** (or dual-basin potential).

This is, in fact, shown in the following figure from an article that inspired this tutorial

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/smogdual_03.jpg'/>
<figcaption>FIGURE 2. Plot of the contacts unique to the closed form of adenylate kinase. Each point represents a contact between residue <i>i</i> and residue <i>j</i> that is unique to the closed form. The Y-axis is the distance between the C${\alpha}$ atoms of residues <i>i</i> and <i>j</i> in the open form and the X-axis is the distance in the closed form. Contacts above the line of slope 1.5 (solid line) constitute the set of contacts selected for the dual-basin SBM models<br> Whitford PC et al (2007) <i> J Mol Biol 366(5), 1661-1671</i></figcaption></center>
</figure>

1. **How can we first filter the contacts that are unique to the closed conformation?** We will use `grep` over the **.contact.CG files** you just obtained during the SMOG2 processing for the open and closed states:

In [ ]:
#Creating a folder for preparing the dual-basin SBM model
dualpath = Path("dualSBM")

#Now, we create the folder using the os.mkdir() command
#The if conditional is just to check whether the folder already exists
#In which case, python returns an error
if os.path.exists(dualpath):
  print("path already exists")
if not os.path.exists(dualpath):
  os.mkdir(dualpath)
  print("path was succesfully created")

#Switching to this new folder
os.chdir(dualpath)

In [ ]:
#Using grep to check which lines in file1 (4ake) are also in file2 (1ake)
#The -v flag inverts the sence of matching, thus printing non-matching lines from file2
!grep -Fxvf ../4ake_smog.contacts.CG ../1ake_smog.contacts.CG > uniquecontacts.txt

If all goes well, you should obtain a list of 113 contacts in the format `chain_i res_i chain_j res_j`. Now, we have to evaluate if the distance of these contacts is significantly different between the open and closed states.

2. To determine the distance difference between these contacts in the open and closed states, we will cheat a little bit. We will first use the `trjconv` module from GROMACS to generate a **coarse-grained PDB file** of both states based in our coarse-grained SBM .gro file. Then, we will use SMOG2, along with these coarse-grained PDB files and our list of unique contacts for the closed structure. With this small trick, we will obtain the **LJ parameters (and therefore the distances!)** of the ligand-induced contacts both in the open (4AKE) and closed (1AKE) states.

In [ ]:
#Generating coarse-grained PDB for 4AKE and 1AKE using GROMACS
%%bash
source /content/gromacs_sbm/bin/GMXRC
gmx editconf -f ../4ake_smog.gro -o 4ake_CA.pdb
gmx editconf -f ../1ake_smog.gro -o 1ake_CA.pdb
#Edit your file to comply with SMOG2 parameters
grep ATOM 4ake_CA.pdb > 4ake_CA_clean.pdb
echo "END" >> 4ake_CA_clean.pdb
grep ATOM 1ake_CA.pdb > 1ake_CA_clean.pdb
echo "END" >> 1ake_CA_clean.pdb

In [ ]:
!udocker --allow-root run --rm -v $(pwd):/workdir smogserver/smog2:stable smog2 -i 4ake_CA_clean.pdb -c uniquecontacts.txt -t /opt/smog2/SBM_calpha -dname 4ake_unique
!udocker --allow-root run --rm -v $(pwd):/workdir smogserver/smog2:stable smog2 -i 1ake_CA_clean.pdb -c uniquecontacts.txt -t /opt/smog2/SBM_calpha -dname 1ake_unique

If we remember from our previous tutorial, the `[ pairs ]` section of the .top file contains the native contacts and their parameters.

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/smogdual_05.png'/>
</center>
</figure>

Since we requested the same user-defined contact map for both files in the previous step, we can get the difference in distance between each contact by **dividing column 4 (or 5) from one file by the column 4 (or 5) from another file**

3. Create two text files on Google Colab in which you will store only the `[ pairs ]` section of these newly generated .top files. Then, we will use `awk` over these files to determine which interactions significantly change between these states:

In [ ]:
#Creating two files only containing the pairs sections using awk
#And eliminating first and last printed lines using sed
!awk 'NR==1, $2 == "pairs" {next}; NF {print $0}; $2 == "exclusions" {exit}' 1ake_unique.top | sed '1d;$d' > closed.pairs
!awk 'NR==1, $2 == "pairs" {next}; NF {print $0}; $2 == "exclusions" {exit}' 4ake_unique.top | sed '1d;$d' > open.pairs

In [ ]:
!paste open.pairs closed.pairs | awk '{if(($4/$9)>1.5)print $1, $2, $3, $9, $10}' > Qligand.pairs

After this, we will obtain a text file with **83 contacts** that are unique to the closed state for which a significant change in distance (>50%) occurs upon reaching the unbound, open conformation.

4. We will also use `awk` to generate the `[ exclusion ]` lists for these ligand-induced native contacts.


In [ ]:
!paste open.pairs closed.pairs | awk '{if(($4/$9)>1.5)print $1, $2}' > Qligand.exclusions

Now we have everything we need to generate a dual-basin model for simulating the conformational change of human adenylate kinase: coordinate and parameter files and ligand-induced native contacts.

5. We will take one of the .top files for one of the states of human adenylate kinase and manually add the ligand-induced pairs and exclusions that we just obtained. Given our previous assumption on the conformational change of this protein, **the most reasonable strategy is to use the .top file from the open (4AKE) structure for its modification.**

In [ ]:
#Copy the .top file for 4AKE into this folder and modify it!
!cp ../4ake_smog.top dualAKE.top

#Part III – Run and analyze our dual-basin SBM simulations

Now, we are ready to perform our simulations of the ligand-induced conformational change of human adenylate kinase using these dual-basin SBM models.

1. We will start by creating a new folder for preparing and running our MD simulations, in which we will copy our SBM coordinate and topology file.

In [ ]:
#Defining a new folder for the MD simulations
mdpath = Path("/content/md_dualAKE/")

#Now, we create the folder using the os.mkdir() command
#The if conditional is just to check whether the folder already exists
#In which case, python returns an error
if os.path.exists(mdpath):
  print("path already exists")
if not os.path.exists(mdpath):
  os.mkdir(mdpath)
  print("path was succesfully created")

In [ ]:
#Changing to our newly created directory and copying the .gro and .top files
os.chdir(mdpath)
from shutil import copyfile
copyfile(smogpath/'4ake_smog.gro', mdpath/'dualAKE.gro')
copyfile(smogpath/dualpath/'dualAKE.top', mdpath/'dualAKE.top')

2. Then, we will download the same **MD instruction file** that we used in our previous tutorial (**mdrun_CA_v5.mdp**), changing the simulation temperature to 108 and the number of steps to 5000000. We will also download the Perl script to generate our LJ 12-10 tabulated potentials.

In [ ]:
%%bash
wget https://github.com/pb3lab/ibm3202/raw/master/files/mdrun_CA_v5.mdp
wget https://github.com/pb3lab/ibm3202/raw/master/files/maketable4.pl
perl maketable4.pl > table.xvg

3. Lastly, we will prepare our **.tpr portable binary run input file for GROMACS** in this folder and run our simulation! Please note how we instruct GROMACS to use our custom table of LJ 12-10 tabulated potentials.

**Let the simulation run for ~15 min** before stopping it (pressing the stop button in the code cell) and analyzing the results.

In [ ]:
#Preparing our binary run input file
%%bash
source /content/gromacs_sbm/bin/GMXRC
gmx grompp -f mdrun_CA_v5.mdp -c dualAKE.gro -p dualAKE.top -o run.tpr

In [ ]:
#Running our simulation
%%time
%%bash
source /content/gromacs_sbm/bin/GMXRC
gmx mdrun -s run.tpr -table table.xvg -tablep table.xvg -nt 2 -noddcheck

4. Once our simulation is done, we can analyze if the conformational change is observed in our trajectory file. For simplification, we will first use the `rmsd` module along with the initial, open structure, as evidence of this change.

In [ ]:
%%bash
source /content/gromacs_sbm/bin/GMXRC
#Commands for RMSD
echo "0" > options
echo " " >> options
echo "0" >> options
echo " " >> options
#RMSD calculation
gmx rms -s dualAKE.gro -f traj_comp.xtc -xvg none < options

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = np.loadtxt('rmsd.xvg')

plt.title('Structural fluctuations of the system')
plt.xlabel('Time (tau)')
plt.ylabel('rmsd (nm)')
plt.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
plt.show()

2. A better metric for looking at the conformational change would be to directly determine if the ligand-induced native contacts are formed. For this, we will use the `g_kuh` module again, along with the list of contacts that are unique to the closed structure and the closed structure as reference for the native distances. Please check how we are generating the contact map file required for this analysis:

In [ ]:
#Generating the contact map file for g_kuh
f = open(mdpath/"Qligand.ndx", "a")
f.write("[ Qligand ]\n")
with open(smogpath/dualpath/"Qligand.exclusions") as infile:
  for line in infile:
    if line.strip():
      cols = line.split()
      f.write(cols[0] + "\t" + cols[1] + "\n")
f.close()

#Copying the reference structure of the closed state for determining native distances
copyfile(smogpath/'1ake_smog.gro', mdpath/'1ake_smog.gro')

In [ ]:
#Analyzing the formation of ligand-induced contacts in our trajectory
%%bash
source /content/gromacs_sbm/bin/GMXRC
g_kuh -s 1ake_smog.gro -f traj_comp.xtc -n Qligand.ndx -noabscut -noshortcut -cut 0.2

4. Let's plot our results and see what happened during our simulation! We will plot first the change in native contacts (Q), and then the change in potential energy. You can check the change in RMSD yourself

In [ ]:
!paste rmsd.xvg qvals.out > data.txt
import matplotlib.pyplot as plt
import numpy as np

data = np.loadtxt('data.txt')

plt.title('Structural fluctuations of the system')
plt.xlabel('Time (tau)')
plt.ylabel('Q')
plt.plot(data[:,0], data[:,2], linestyle='solid', linewidth='2', color='red')
plt.show()

5. To finalize, we will visualize our simulation. For this, we will use the `trjconv` module to extract only the protein from our system and convert our trajectory into a PDB file and then  download this new PDB file and load it onto [**NGLviewer**](http://nglviewer.org/ngl/) as a **trajectory** PDB file.

In [ ]:
%%bash
source /content/gromacs_sbm/bin/GMXRC
#This is a trick to provide interactive options to gmx
echo "Protein" > options
echo "Protein" >> options
echo " "
gmx trjconv -s run.tpr -f traj_comp.xtc -o traj.pdb < options

In [ ]:
#Downloading the trajectory PDB file
from google.colab import files
files.download("/content/md_dualAKE/traj.pdb")

**And this is the end of the tenth tutorial!** Good science!